In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


In [7]:
!pip install jellyfish
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.9 MB/s eta 0:00:00


In [151]:
!pip install fuzzy

  Preparing metadata (setup.py) ... done
  Created wheel for fuzzy: filename=Fuzzy-1.2.2-cp310-cp310-linux_x86_64.whl size=208412 sha256=54a596395893da0e367a4fdc8890b4691801dad66c43fc81b2a5afc601c53b59
  Stored in directory: /root/.cache/pip/wheels/3e/a7/03/cb9f418321ad60736caa7a86f0077f6ab74b669885c3e03cc1
Successfully built fuzzy


In [3]:
filepath = '/content/drive/MyDrive/Colab Notebooks/DATASETS/NAMES/'

In [172]:
import string
import pickle
import jellyfish
import pandas as pd
from rapidfuzz import fuzz
from fuzzy import DMetaphone


import warnings
warnings.filterwarnings('ignore')

In [17]:
valid_names = pd.read_csv(filepath + 'positive_names.csv')

In [19]:
print(valid_names.info())
print(valid_names.isna().sum())
valid_names.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 644322 entries, 0 to 644321
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Name    644313 non-null  object
dtypes: object(1)
memory usage: 4.9+ MB
None
Name    9
dtype: int64


,Name
0,LITER RIBA
1,LINYA ANGO
2,MOGAM PADU
3,MINU BAGRA
4,MINYIR ORI


In [20]:
valid_names['Name'] = valid_names[valid_names['Name'].notna()]

In [21]:
valid_names.sample(6)

,Name
357766,BIMLA DEVI
231807,MINAKSHI ANNASAHEB KOLHE
503401,JITENDER PAL SINGH
222215,PRITI DHANKHER
622369,SHAMBARAKUMARAN P
427545,VEERANNA


In [22]:
valid_names.shape[0]

644322

In [23]:
valid_names['Name'] = valid_names['Name'].str.split(' ')

In [87]:
name_data = valid_names.explode('Name')
name_data.reset_index(drop=True, inplace=True)

In [88]:
name_data.head()

,Name
0,LITER
1,RIBA
2,LINYA
3,ANGO
4,MOGAM


In [89]:
name_data.shape[0]

1430463

In [90]:
name_data = name_data[name_data['Name'].str.len() > 3]

In [91]:
name_data.shape[0]

1217917

In [92]:
ap_data =pd.read_csv(filepath + 'ap_names.txt', delimiter='\t')

In [93]:
ap_data.head()

,year,#people_name_born_in_year,#male_name/year,#female_name/year,name
0,1924,1,1,0,aa
1,1934,1,1,0,aa
2,1937,1,0,1,aa
3,1938,3,2,1,aa
4,1941,1,1,0,aa


In [94]:
ap_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12420176 entries, 0 to 12420175
Data columns (total 5 columns):
 #   Column                     Dtype 
---  ------                     ----- 
 0   year                       int64 
 1   #people_name_born_in_year  int64 
 2   #male_name/year            int64 
 3   #female_name/year          int64 
 4   name                       object
dtypes: int64(4), object(1)
memory usage: 473.8+ MB


In [95]:
ap_data['name'].duplicated().value_counts()

True     9529558
False    2890618
Name: name, dtype: int64

In [96]:
ap_data.drop_duplicates(subset='name', inplace=True)

In [97]:
ap_data.shape[0]

2890618

In [98]:
ap_data.head()

,year,#people_name_born_in_year,#male_name/year,#female_name/year,name
0,1924,1,1,0,aa
58,1982,1,1,0,aaa
59,1994,1,1,0,aaaa
60,1985,1,1,0,aaabkhan
61,1983,1,0,1,aaachamma


In [99]:
ap_data = ap_data[ap_data['name'].str.len() > 3]

In [100]:
ap_data.head()

,year,#people_name_born_in_year,#male_name/year,#female_name/year,name
59,1994,1,1,0,aaaa
60,1985,1,1,0,aaabkhan
61,1983,1,0,1,aaachamma
62,1985,1,1,0,aaadhi
63,1966,1,1,0,aaadhinarayana


In [101]:
mask = ap_data['name'].str.contains(r'^(.)\1+$')
ap_data_filtered = ap_data[~mask]
ap_data_filtered.shape[0]

2884928

In [102]:
ap_data_filtered.head()

,year,#people_name_born_in_year,#male_name/year,#female_name/year,name
60,1985,1,1,0,aaabkhan
61,1983,1,0,1,aaachamma
62,1985,1,1,0,aaadhi
63,1966,1,1,0,aaadhinarayana
64,1972,1,0,1,aaadi


In [103]:
ap_data_filtered.rename(columns = {"name":"Name"}, inplace = True)

In [104]:
ap_data_filtered.drop(columns = ['year', '#people_name_born_in_year', '#male_name/year', '#female_name/year'], inplace = True)

In [105]:
ap_data_filtered.head()

,Name
60,aaabkhan
61,aaachamma
62,aaadhi
63,aaadhinarayana
64,aaadi


In [106]:
name_data['Name'] = name_data['Name'].str.lower()

In [110]:
name_data.head()

,Name
0,liter
1,riba
2,linya
3,ango
4,mogam


In [109]:
ap_data_filtered.head()

,Name
60,aaabkhan
61,aaachamma
62,aaadhi
63,aaadhinarayana
64,aaadi


In [111]:
last_index = name_data.index[-1]
ap_data_filtered.reset_index(drop=True, inplace=True)
combined_data = pd.concat([name_data, ap_data_filtered], ignore_index=True)

In [112]:
combined_data.shape[0]

4102845

In [113]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4102845 entries, 0 to 4102844
Data columns (total 1 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   Name    object
dtypes: object(1)
memory usage: 31.3+ MB


In [114]:
combined_data.head()

,Name
0,liter
1,riba
2,linya
3,ango
4,mogam


In [115]:
combined_data = combined_data.sample(frac = 1)

In [120]:
combined_data.head()

,Name
3983758,verikela
2436809,konagoti
2624198,maishabhoyi
1009925,shailender
3232640,raghumanadala


In [121]:
combined_data['Phonetic'] = combined_data['Name'].apply(jellyfish.soundex)

In [152]:
encoder = DMetaphone()

combined_data['Dmeta'] = combined_data['Name'].apply(lambda name: encoder(name)[0])


In [178]:
combined_data.head()

,Name,Dmeta
3983758,verikela,b'FRKL'
2436809,konagoti,b'KNKT'
2624198,maishabhoyi,b'MXP'
1009925,shailender,b'XLNT'
3232640,raghumanadala,b'RKMN'


In [179]:
combined_data.to_csv(filepath + "combined_name_data130723.csv")

In [173]:
def validate_names(full_name):
    salutations = ["mr", "ms", "mrs", "dr", "sir", "master", "madam", "maam", "prof", "rev", "capt", "major","col"]

    names = full_name.lower().split()
    names = [name for name in names if name not in salutations]
    names = [name.translate(str.maketrans('', '', string.punctuation)) for name in names]


    for name in names:
        if any(char.isdigit() for char in name):
            return "INVALID"

        if len(name) <= 2 or len(name) >= 20:
            return "INVALID"

        if not any(char.lower() in 'aeiou' for char in name):
            return "INVALID"

    for name in names:
        primary_code, _ = encoder(name)

        if name in combined_data['Name'].unique().tolist():
            print(f"Matched name: {name}")
            return "VALID"

        if primary_code in combined_data['Phonetic'].unique().tolist():
            print(f"Matched phonetic: {name}")
            return "VALID"

    return "INVALID"


In [174]:
name = 'dr varun'
validate_names(name)

Matched name: varun


'VALID'